In [ ]:
import os
import xml.etree.ElementTree as ET

# Define dataset and output paths
DATASET_DIR = r"E:\vechile_project\Final Train Dataset" # folder containing 1.jpg, 1.xml, 2.jpg, 2.xml, etc.
OUT_LABEL_DIR = r"E:\vechile_project\labels"              # folder to save YOLO txt files
os.makedirs(OUT_LABEL_DIR, exist_ok=True)

# Mapping from various annotation labels to your simplified classes
map_to = {
    "car": "car", "suv": "car", "taxi": "car",
    "bus": "bus", "van": "van", "pickup": "truck",
    "motorbike": "motorbike", "three wheelers (CNG)": "three_wheeler"
}

# Final classes you want to detect, order defines class IDs
final_classes = ["car", "bus", "motorbike", "three_wheeler", "truck", "van"]
class_to_id = {c: i for i, c in enumerate(final_classes)}


def convert(xml_path):
    try:
        tree = ET.parse(xml_path)
    except ET.ParseError as e:
        print(f"Parse error in file {xml_path}: {e}")
        return []
    
    root = tree.getroot()
    w = int(root.find("size/width").text)
    h = int(root.find("size/height").text)
    yolo_lines = []
    
    for obj in root.findall("object"):
        name = obj.find("name").text.strip()
        name = map_to.get(name)
        if name is None or name not in class_to_id:
            continue
        cls_id = class_to_id[name]
        b = obj.find("bndbox")
        xmin = float(b.find("xmin").text)
        ymin = float(b.find("ymin").text)
        xmax = float(b.find("xmax").text)
        ymax = float(b.find("ymax").text)

        xmin, ymin = max(0, xmin), max(0, ymin)
        xmax, ymax = min(w, xmax), min(h, ymax)
        if xmax <= xmin or ymax <= ymin:
            continue

        x_c = ((xmin + xmax) / 2.0) / w
        y_c = ((ymin + ymax) / 2.0) / h
        bw = (xmax - xmin) / w
        bh = (ymax - ymin) / h

        yolo_lines.append(f"{cls_id} {x_c:.6f} {y_c:.6f} {bw:.6f} {bh:.6f}")
    
    return yolo_lines


In [20]:
import os
import shutil
import random
from pathlib import Path


DATASET_DIR = r"E:\vechile_project\Final Train Dataset"
LBL_DIR = r"E:\vechile_project\Final Train Dataset\labels"
OUT = r"E:\vechile_project\dataset_yolo"


# Create output folders
for folder in ["images/train", "images/val", "labels/train", "labels/val"]:
    Path(os.path.join(OUT, folder)).mkdir(parents=True, exist_ok=True)

# List all images in dataset folder (jpg, jpeg, png)
images = [f for f in os.listdir(DATASET_DIR) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

random.seed(42)
random.shuffle(images)

split = int(len(images) * 0.8)
train_imgs = images[:split]
val_imgs = images[split:]

def copy_list(img_list, split_name):
    for img in img_list:
        base = os.path.splitext(img)[0]

        src_img = os.path.join(DATASET_DIR, img)
        src_lbl = os.path.join(LBL_DIR, base + ".txt")

        dst_img = os.path.join(OUT, f"images/{split_name}", img)
        dst_lbl = os.path.join(OUT, f"labels/{split_name}", base + ".txt")

        shutil.copy2(src_img, dst_img)

        if os.path.exists(src_lbl):
            shutil.copy2(src_lbl, dst_lbl)
        else:
            # Create empty label file if not found
            open(dst_lbl, "w").close()

copy_list(train_imgs, "train")
copy_list(val_imgs, "val")

print(f"Split done. Train: {len(train_imgs)} Val: {len(val_imgs)}")


Split done. Train: 2402 Val: 601


In [20]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")
model.train(
    data='dataset_yolo/dataset.yaml',
    epochs=15,
    imgsz=320,
    batch=4,
    name='veh_exp_fast',
    val=False 
)


Ultralytics 8.3.177  Python-3.9.21 torch-2.8.0+cpu CPU (12th Gen Intel Core(TM) i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset_yolo/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=veh_exp_fast, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, p

train: Scanning E:\vechile_project\dataset_yolo\labels\train... 2360 images, 2276 backgrounds, 42 corrupt: 100%|██████████| 2402/2402 [00:01<00:00, 1322.02it/s]

train: E:\vechile_project\dataset_yolo\images\train\144.jpg: ignoring corrupt image/label: 
train: E:\vechile_project\dataset_yolo\images\train\145.jpg: corrupt JPEG restored and saved
train: E:\vechile_project\dataset_yolo\images\train\147.jpg: corrupt JPEG restored and saved
train: E:\vechile_project\dataset_yolo\images\train\148.jpg: ignoring corrupt image/label: 
train: E:\vechile_project\dataset_yolo\images\train\149.jpg: ignoring corrupt image/label: 
train: E:\vechile_project\dataset_yolo\images\train\150.jpg: ignoring corrupt image/label: 
train: E:\vechile_project\dataset_yolo\images\train\151.jpg: ignoring corrupt image/label: 
train: E:\vechile_project\dataset_yolo\images\train\152.jpg: ignoring corrupt image/label: 
train: E:\vechile_project\dataset_yolo\images\train\153.jpg: ignoring corrupt image/label: 
train: E:\vechile_project\dataset_yolo\images\train\154.jpg: ignoring corrupt image/label: 
train: E:\vechile_project\dataset_yolo\images\train\155.jpg: ignoring corrupt 

train: New cache created: E:\vechile_project\dataset_yolo\labels\train.cache
val: Fast image access  (ping: 0.20.2 ms, read: 298.7316.1 MB/s, size: 684.4 KB)


e:\vechile_project\vechileenv\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning E:\vechile_project\dataset_yolo\labels\val.cache... 601 images, 575 backgrounds, 0 corrupt: 100%|██████████| 601/601 [00:00<?, ?it/s]

...

      15/15         0G     0.2698     0.7621     0.1748          0        320: 100%|██████████| 590/590 [10:17<00:00,  1.05s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 76/76 [00:45<00:00,  1.66it/s]


                   all        601        402     0.0492      0.122     0.0429     0.0128

15 epochs completed in 2.792 hours.
Optimizer stripped from runs\detect\veh_exp_fast\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\veh_exp_fast\weights\best.pt, 6.2MB

Validating runs\detect\veh_exp_fast\weights\best.pt...
Ultralytics 8.3.177  Python-3.9.21 torch-2.8.0+cpu CPU (12th Gen Intel Core(TM) i5-1235U)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 76/76 [00:35<00:00,  2.13it/s]


                   all        601        402     0.0493      0.123     0.0429     0.0128
                   car         20        172     0.0297      0.436     0.0904     0.0392
                   bus         15         89     0.0159      0.281     0.0398      0.012
             motorbike         17         52       0.25     0.0192      0.127     0.0255
         three_wheeler         11         51          0          0          0          0
                 truck         10         23          0          0          0          0
                   van          8         15          0          0          0          0
Speed: 0.8ms preprocess, 33.4ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\veh_exp_fast


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002D0A7500340>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
 

In [22]:
import cv2


model = YOLO("runs/detect/veh_exp_fast/weights/best.pt")

results = model.predict(source="E:\\vechile_project\\dataset_yolo\\images\\val\\26.jpg", conf=0.05, imgsz=640)

# results is a list, one item per image
for result in results:
    img = result.orig_img.copy()  # original image numpy array
    
    boxes = result.boxes
    if boxes is None or len(boxes) == 0:
        print("No detections found in this image")
    else:
        for box in boxes:
            # box.xyxy gives xmin,ymin,xmax,ymax
            xmin, ymin, xmax, ymax = box.xyxy[0].cpu().numpy().astype(int)
            conf = box.conf[0].cpu().numpy()
            cls_id = int(box.cls[0].cpu().numpy())
            label = model.names[cls_id]

            # Draw rectangle and label
            cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            cv2.putText(img, f"{label} {conf:.2f}", (xmin, ymin - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

        # Save or display image
        out_path = "result_02.jpg"
        cv2.imwrite(out_path, img)
        print(f"Saved result image to {out_path}")
        cv2.imshow("Detections", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()



image 1/1 E:\vechile_project\dataset_yolo\images\val\26.jpg: 640x640 (no detections), 389.9ms
Speed: 28.1ms preprocess, 389.9ms inference, 16.6ms postprocess per image at shape (1, 3, 640, 640)
No detections found in this image


In [23]:
results = model.predict(
    source="E:/vechile_project/dataset_yolo/images/val",  # folder path
    save=True,
    imgsz=320
)


image 1/601 E:\vechile_project\dataset_yolo\images\val\02.jpg: 224x320 (no detections), 125.9ms
image 2/601 E:\vechile_project\dataset_yolo\images\val\03.jpg: 192x320 (no detections), 137.2ms
image 3/601 E:\vechile_project\dataset_yolo\images\val\07.jpg: 256x320 (no detections), 80.8ms
image 4/601 E:\vechile_project\dataset_yolo\images\val\09.jpg: 192x320 (no detections), 36.4ms
image 5/601 E:\vechile_project\dataset_yolo\images\val\103.jpg: 224x320 (no detections), 44.6ms
image 6/601 E:\vechile_project\dataset_yolo\images\val\104.jpg: 320x256 (no detections), 83.0ms
image 7/601 E:\vechile_project\dataset_yolo\images\val\111.jpg: 320x256 (no detections), 31.7ms
image 8/601 E:\vechile_project\dataset_yolo\images\val\115.jpg: 320x256 (no detections), 35.9ms
image 9/601 E:\vechile_project\dataset_yolo\images\val\118.jpg: 320x256 (no detections), 41.3ms
image 10/601 E:\vechile_project\dataset_yolo\images\val\120.jpg: 320x192 (no detections), 127.0ms
image 11/601 E:\vechile_project\dataset

error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 6220800 bytes in function 'cv::OutOfMemoryError'
